In [1]:
from collections import Counter, OrderedDict
import numpy as np
import string
import re
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Dropout, Embedding
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences 
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import BatchNormalization
from keras.layers.recurrent import LSTM
from keras import optimizers
import tensorflow as tf
import pandas as pd
# import pickle
# from spacy.vocab import Vocab
# from spacy.language import Language
# import spacy

Using TensorFlow backend.


In [22]:
#making files
from_lang='bengali'
to_lang='english' 


In [23]:
def data(from_lang):
    
    punc_table = str.maketrans({key: None for key in string.punctuation})
    sentences = []
    targets = []
    count =0;
    
    with open('dataset/'+str(from_lang)+'_word_def.txt', 'r') as filee:
        
        for i, line in enumerate(filee):
    #     for line in dataset:
#             count+=1
            words = line.strip('\n').split('\t')
#             words = line
            word = words[0]
            definitions = words[1].split(';')
            for definition in definitions:
#                 print(definition)
    # Removing data between () in definiton example to look at(something) => to look at
                definition = re.sub("[\(\[].*?[\)\]]", "", definition).replace('  ', ' ')
    # Removing the last extra space from the definition
#                 if definition[-1] == ' ':
#                     definition = definition[:-1]
    # To lower case and removing punctuation
                temp_word_list = definition.translate(punc_table).lower().split(' ')
                temp_word_list = list(filter(None, temp_word_list))
                mid_sent=['<start>'] + temp_word_list + ['<end>']
                if mid_sent in sentences:
                    continue
                sentences.append(['<start>'] + temp_word_list + ['<end>'])

                targets.append(word)
    
#     words = [word for word_sublist in sentences for word in word_sublist]
    WORD=[]
    for word_sublist in sentences:
        for word in word_sublist:
            WORD.append(word)
    words=WORD
    inf = float('inf')
    frequency_dict = OrderedDict({'<end>': inf, '<start>': inf})
    words_frequency_dict = sorted(Counter(words).most_common(None), key=lambda x:x[1], reverse=True)
    
    defs_frequency_dict = sorted(Counter(targets).most_common(None), key=lambda x:x[1], reverse=True)
    frequency_dict.update(words_frequency_dict)
    frequency_dict.update(defs_frequency_dict)
    frequency_dict.move_to_end('<start>', last=False)

    word2idx = OrderedDict([(item[0], i) for i,item in enumerate(frequency_dict.items())])
    idx2word = dict(zip(word2idx.values(), word2idx.keys()))
    
    return sentences, targets, word2idx, idx2word

In [24]:
def sent2idx(sentences, targets, word2idx):
    s = []
    for sentence in sentences:
        s.append([word2idx[word] for word in sentence])
    t = []
    for word in targets:
        t.append(word2idx[word])
    return s, t

In [25]:
SEQUENCE_LENGTH = 100
BATCH_SIZE = 100
EPOCHS = 100

sentences, targets, word2idx, idx2word = data(from_lang)
sentences, targets = sent2idx(sentences, targets, word2idx)
targets = np_utils.to_categorical(targets)
batches = len(sentences) // BATCH_SIZE
print('Sentences:',len(sentences))
print('Vocab Size:',len(word2idx))
print('Batches:',batches)

Sentences: 9262
Vocab Size: 17455
Batches: 92


In [26]:
model = Sequential()
model.add(Embedding(len(word2idx),100))
model.add(BatchNormalization())
model.add(LSTM(128, return_sequences=False))
model.add(BatchNormalization())
model.add(Dense(len(word2idx), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, None, 100)         1745500   
_________________________________________________________________
batch_normalization_9 (Batch (None, None, 100)         400       
_________________________________________________________________
lstm_5 (LSTM)                (None, 128)               117248    
_________________________________________________________________
batch_normalization_10 (Batc (None, 128)               512       
_________________________________________________________________
dense_5 (Dense)              (None, 17455)             2251695   
Total params: 4,115,355
Trainable params: 4,114,899
Non-trainable params: 456
_________________________________________________________________
None


In [7]:
def generator(inputs, targets, batch_size):
    inputs = np.array(inputs)
    while True:
        perm = np.random.permutation(len(inputs))[:batch_size]
#         print(perm)
        batch_inputs = pad_sequences(inputs[perm])
        batch_targets = targets[perm]
#         print(len(batch_targets[0]),len(targets[0]),)
#         batch_targets = np_utils.to_categorical(batch_targets)
#         print(type(batch_targets))
#         print(len(batch_targets),len(batch_inputs))
        yield batch_inputs, batch_targets

In [8]:
input_ = np.array(targets)
# targets = input_.reshape(3281,5817, 1)
print(len(sentences[0]))

5


In [9]:
model.fit_generator(generator = generator(sentences, targets, BATCH_SIZE), epochs = 100, samples_per_epoch = 60)

/home/divyani/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(generator=<generator..., epochs=100, steps_per_epoch=60)`
  """Entry point for launching an IPython kernel.



Epoch 1/100
60/60 [==============================] - 12s 201ms/step - loss: 9.6122
Epoch 2/100
60/60 [==============================] - 9s 145ms/step - loss: 9.0319
Epoch 3/100
60/60 [==============================] - 8s 135ms/step - loss: 8.2028
Epoch 4/100
60/60 [==============================] - 9s 146ms/step - loss: 7.1610
Epoch 5/100
60/60 [==============================] - 10s 173ms/step - loss: 6.1264
Epoch 6/100
60/60 [==============================] - 9s 155ms/step - loss: 4.9351
Epoch 7/100
60/60 [==============================] - 10s 167ms/step - loss: 3.8398
Epoch 8/100
60/60 [==============================] - 9s 143ms/step - loss: 2.8944
Epoch 9/100
60/60 [==============================] - 9s 143ms/step - loss: 2.0673
Epoch 10/100
60/60 [==============================] - 8s 134ms/step - loss: 1.4285
Epoch 11/100
60/60 [==============================] - 9s 146ms/step - loss: 0.9381
Epoch 12/100
60/60 [==============================] - 9s 153ms/step - loss: 0.6390
Epoch 13/

In [10]:
model.save('model/bengali.h5')

In [12]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
model = load_model('model/bengali.h5')
# Check its architecture
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         1745500   
_________________________________________________________________
batch_normalization_1 (Batch (None, None, 100)         400       
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
dense_1 (Dense)              (None, 17455)             2251695   
Total params: 4,115,355
Trainable params: 4,114,899
Non-trainable params: 456
________________________________________________________

In [64]:
test_label=[]
result_label=[]
with open('TEST.txt', 'r') as filee:
    for i, line in enumerate(filee):
        pair = line.strip('\n').split('\t')
        word = pair[0]
        test_label.append(word)
        definition = pair[1]
        words = definition.split(' ')
        idxs = []
        for word in words:
#             print(word)
            if(word in word2idx):
                idxs.append(word2idx[word])

        idxs = np.array([0] + idxs + [1]).reshape((1,len(idxs) + 2))
        prediction = model.predict(idxs, verbose=0)
        index = np.argmax(prediction)
        meaning = idx2word[index]
        result_label.append(meaning)


FileNotFoundError: [Errno 2] No such file or directory: 'TEST.txt'

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(test_label, result_label))

In [25]:
definition = 'a fabric used for suits'
words = definition.split(' ')
idxs = []
for word in words:
#             print(word)
    if(word in word2idx):
        idxs.append(word2idx[word])

idxs = np.array([0] + idxs + [1]).reshape((1,len(idxs) + 2))
prediction = model.predict(idxs, verbose=0)
index = np.argmax(prediction)
meaning = idx2word[index]
print(meaning)

suiting
